In [ ]:
"""
A minimal implementation of Monte Carlo tree search (MCTS) in Python 3
Luke Harold Miles, July 2019, Public Domain Dedication
See also https://en.wikipedia.org/wiki/Monte_Carlo_tree_search
https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1
"""
"""
Pulled from the web to be examined and modified
"""

from abc import ABC, abstractmethod
from collections import defaultdict
import math


class MCTS:
    "Monte Carlo tree searcher. First rollout the tree then choose a move."

    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score)

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        self.children[node] = node.find_children()

    def _simulate(self, node):
        "Returns the reward for a random simulation (to completion) of `node`"
        invert_reward = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                return 1 - reward if invert_reward else reward
            node = node.find_random_child()
            invert_reward = not invert_reward

    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = 1 - reward  # 1 for me is 0 for my enemy, and vice versa

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            "Upper confidence bound for trees"
            return self.Q[n] / self.N[n] + self.exploration_weight * math.sqrt(
                log_N_vertex / self.N[n]
            )

        return max(self.children[node], key=uct)


class Node(ABC):
    """
    A representation of a single board state.
    MCTS works by constructing a tree of these Nodes.
    Could be e.g. a chess or checkers board state.
    """
    def __init__(self, state, parent = None, parent_action = None):
        #state will be a 2 by 2 array, the top two representing the opp, the
        #bottom two representing the player
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children = []
        self._num_visits = 0
        self._results = defaultdict(int)
        self._results[1] = 0
        self._results[-1] = 0
        self._untried_actions = None
        self._untried_actions = self.untried_actions()
        return
    
    def untried_actions(self):
        self._untried_actions = self.state.get_legal_actions()
        return self._untried_actions
    
    def q(self):
        wins = self._results[1]
        loses = self._results[-1]
        return wins - loses
    
    def n(self):
        return self._num_visits
    
    #from present state, an action is chosen and the next state is generated based on that action
    #the child node is created with the state of "next state" and appended to the list of children
    #of the current node
    def expand(self):
        action = self._untried_actions.pop()
        next_state = self.state.move(action)
        child_node = Node(next_state, parent = self, parent_action = action)
        
        self.children.append(child_node)
        return child_node
    
    def is_terminal_node(self):
        return self.state.is_game_over()
    
    #for our application of MCTS, this step would be to run it through the neural network
    #rather than do a full simulation
    def rollout(self, result):
        self._num_visits += 1
        self._results[result] += 1
        if self.parent:
            self.parent.backpropagate(result)
        return
    
    def backpropogate(self):
        self._num_visits += 1.
        self._results[result] += 1.
        if self.parent:
            self.parent.backpropagate(result)
        return
    
    def is_fully_expanded(self):
        return len(self._untried_actions) == 0
    
    def best_child(self, c_param = 0.1):
        choices_weights = [(c.q() / c.n()) + c_param * 
                           np.sqrt((2 * np.log(self.n()) / c.n())) for c in self.children]
        return self.children[np.argmax(choices_weights)]
    
    #randomly selects a next move (not necessary for our implementation)
    def rollout_policy(self, possible_moves):
        return possible_moves[np.random.randint(len(possible_moves))]
    
    def _tree_policy(self):
        current_node = self
        while not current_node.is_terminal_state():
            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node
    
    def best_action(self):
        simulation_no = 100
        
        for i in range(simulation_no):
            v = self._tree_policy()
            #will have to change this for our implementation
            reward = v.rollout()
            v.backpropagate(reward)
    
    def get_legal_actions(self):
        
    
    

    @abstractmethod
    def find_children(self):
        "All possible successors of this board state"
        return set()

    @abstractmethod
    def find_random_child(self):
        "Random successor of this board state (for more efficient simulation)"
        return None

    @abstractmethod
    def is_terminal(self):
        "Returns True if the node has no children"
        return True

    @abstractmethod
    def reward(self):
        "Assumes `self` is terminal node. 1=win, 0=loss, .5=tie, etc"
        return 0

    @abstractmethod
    def __hash__(self):
        "Nodes must be hashable"
        return 123456789

    @abstractmethod
    def __eq__(node1, node2):
        "Nodes must be comparable"
        return True

In [ ]:
class State:
    #states will be held in 
    def __init__(self):
        state = defaultdict(list)
        state['player'] = [1, 1]
        state['opp'] = [1, 1]
        return
    
    #mutator that takes the player_side (left or right) and the opp_side they want to act on
    def player_move(self, plr_side, opp_side):
        plr_side = -1
        if plr_side == 'right':
            plr_ind = 1
        elif plr_side == 'left':
            plr_ind = 0
        else:
            raise Exception("plr_side and opp_side arguments can only accept 'right' or 'left as input'")
        
        

In [ ]:
class Play_Chop:
    def __init__(self):
        
    

In [ ]:
class Game_Chop:
    #Generate the initial game state.
    def __init__(self):
        #TODO
        
    
    #return current players legal moves from a given state 
    #(similar if not equivalent to findChildren)
    def legalPlays(state):
        #TODO
        return plays
    
    #Advance the given state and return it
    def nextState(state, move):
        #TODO
        return newState
    
    #return the reward from the game, i.e. reward, 1 if win, 0 if loss (can't tie in chopsticks)
    def reward(state):
        #TODO
        return winner
    
    